### Nettoyage : suppression ou traitement des valeurs manquantes et incohérentes.

In [13]:
import pandas as pd
import os

In [30]:
raw_teams_path = '../data/raw/teams'
processed_teams_path = '../data/processed/teams'

teams = os.listdir(raw_teams_path)

- Nettoyage de table joueurs par: la suppression de lignes dont MP = 0 et conversion des colonnes en entiers 

In [ ]:
for team_name in teams:
    df = pd.read_csv(os.path.join(raw_teams_path, team_name) + '/players.csv')

    df = df[df['MP'] != 0]

    df['Min'] = df['Min'].apply(lambda x: int(x.replace(',', '')))
    df['Gls'] = df['Gls'].apply(lambda x: int(x))
    df['Ast'] = df['Ast'].apply(lambda x: int(x))
    df['G+A'] = df['G+A'].apply(lambda x: int(x))
    df['G-PK'] = df['G-PK'].apply(lambda x: int(x))
    df['PK'] = df['PK'].apply(lambda x: int(x))
    df['PKatt'] = df['PKatt'].apply(lambda x: int(x))
    df['CrdY'] = df['CrdY'].apply(lambda x: int(x))
    df['CrdR'] = df['CrdR'].apply(lambda x: int(x))

    df = df[~df.duplicated()]

    os.makedirs(os.path.join(processed_teams_path, team_name), exist_ok=True)

    df.to_csv(os.path.join(processed_teams_path, team_name) + '/players.csv', index=False)


- Nettoyage de table matches

In [97]:
for team_name in teams:
    print(team_name)
    df = pd.read_csv(os.path.join(raw_teams_path, team_name) + '/matches.csv')

    # Garde la premiere partie (si le temps a plusieurs parties)
    df['Time'] = df['Time'].apply(lambda x: x.split(' ')[0])

    # Dans le cas d'un match nul, GA et GF contient le nombre de buts marqués aux tirs au but
    if (df.dtypes['GF'] == 'object'):
        df['GF'] = df['GF'].apply(lambda x: int(x.split(' ')[0]))

    if (df.dtypes['GA'] == 'object'):
        df['GA'] = df['GA'].apply(lambda x: int(x.split(' ')[0]))
    
    # Le nom de l'adversaire parfois contient des caracteres supplémentaire au debut
    df['Opponent'] = df['Opponent'].apply(lambda x: x[3:] if len(x.split(' ')[0]) == 2 else x)

    # Supprimer la virgule et convertir en entier
    df['Attendance'] = df['Attendance'].apply(lambda x: int(x.replace(',', '')) if type(x) == str else x)

    # Supprimer le caractère supplémentaire s'il existe
    df['Formation'] = df['Formation'].apply(lambda x: x.replace('◆', ''))
    df['Opp Formation'] = df['Opp Formation'].apply(lambda x: x.replace('◆', ''))
    
    # Remplacer les valeurs manquantes
    df['Attendance'] = df['Attendance'].fillna(int(df['Attendance'].median()))
    df['Poss'] = df['Poss'].fillna(df['Poss'].median())
    df['xG'] = df['xG'].fillna(round(df['xG'].mean(), 1))
    df['xGA'] = df['xGA'].fillna(round(df['xGA'].mean(), 1))

    df['Attendance'] = df['Attendance'].apply(lambda x: int(x))
    df['Poss'] = df['Poss'].apply(lambda x: int(x))

    df = df[~df.duplicated()]

    os.makedirs(os.path.join(processed_teams_path, team_name), exist_ok=True)

    df.to_csv(os.path.join(processed_teams_path, team_name) + '/matches.csv', index=False)


Arsenal
Aston Villa
Bournemouth
Brentford
Brighton
Chelsea
Crystal Palace
Everton
Fulham
Ipswich Town
Leicester City
Liverpool
Manchester City
Manchester Utd
Newcastle Utd
Nott'ham Forest
Southampton
Tottenham
West Ham
Wolves
